# <b>Excel Data ETL

In [10]:
import pandas as pd
import sqlite3
import os

## <b> Extract Data

In [11]:
def extract_data_from_files(directory):
    data_frames = []
    for file in os.listdir(directory):
        # Check if the file is a CSV or Excel file
        if file.endswith(".xlsx") or file.endswith(".xls") or file.endswith(".csv"):
            file_path = os.path.join(directory, file)
            if file.endswith(".csv"):
                df = pd.read_csv(file_path)
            else:
                df = pd.read_excel(file_path)
            data_frames.append(df)
    return data_frames

## <b> Transform Data

In [12]:
def transform_data(data_frames):
    # Combine all dataframes into one
    combined_df = pd.concat(data_frames, ignore_index=True)
    
    # Example transformations:
    # Remove duplicates
    combined_df = combined_df.drop_duplicates()
    
    # Handle missing values by filling forward then backward
    combined_df = combined_df.ffill().bfill()
    
    # Convert any necessary columns to appropriate data types, if needed
    if 'Population' in combined_df.columns:
        combined_df['Population'] = combined_df['Population'].astype(int)
    
    # Ensure all column names are consistent and lowercase
    combined_df.columns = [col.lower().replace(' ', '_') for col in combined_df.columns]
    
    # Drop duplicate columns if any
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
    
    return combined_df

## <b> Load Data


In [13]:
def load_data_to_db(combined_df, db_name, table_name):
    conn = sqlite3.connect(db_name)
    combined_df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

## <b>Main Function

In [14]:
# Define the directory containing the Excel and CSV files
directory = "Population_Data"

# Define the SQLite database name
database_name = "world_population.db"

# Define the table name in the database
table_name = "world_population_data"

# Run the ETL process
data_frames = extract_data_from_files(directory)
transformed_data = transform_data(data_frames)
load_data_to_db(transformed_data, database_name, table_name)

# Display the first few rows of the transformed data
transformed_data.head()

,#,country_(or_dependency),population_(2022),yearly_change,net_change,density_(p/km²),land_area_(km²),migrants_(net),fert._rate,med.age,...,2013,2014,2016,2017,2018,2019,2021,2023_population,density_(km²),world_percentage
0,1.0,China,"1,439,323,776",0.39%,"5,540,090",153,"9,388,211","-348,399",1.7,38,...,31541209.0,32716210.0,34636207.0,35643418.0,36686784.0,37769499.0,40099462.0,1.428628e+09,481.0,17.85%
1,2.0,India,"1,380,004,385",0.99%,"13,586,631",464,"2,973,190","-532,687",2.2,28,...,31541209.0,32716210.0,34636207.0,35643418.0,36686784.0,37769499.0,40099462.0,1.428628e+09,481.0,17.85%
2,3.0,United States,"331,002,651",0.59%,"1,937,734",36,"9,147,420","954,806",1.8,38,...,31541209.0,32716210.0,34636207.0,35643418.0,36686784.0,37769499.0,40099462.0,1.428628e+09,481.0,17.85%
3,4.0,Indonesia,"273,523,615",1.07%,"2,898,047",151,"1,811,570","-98,955",2.3,30,...,31541209.0,32716210.0,34636207.0,35643418.0,36686784.0,37769499.0,40099462.0,1.428628e+09,481.0,17.85%
4,5.0,Pakistan,"220,892,340",2.00%,"4,327,022",287,"770,880","-233,379",3.6,23,...,31541209.0,32716210.0,34636207.0,35643418.0,36686784.0,37769499.0,40099462.0,1.428628e+09,481.0,17.85%
